##### Instalar requirements:

# 📰 Tarea 1: Sistema de recuperación de noticias (RPP)


In [1]:
#pip install -r requirements.txt

In [ ]:
#pip uninstall langchain langchain-community -y

Found existing installation: langchain 1.0.0
Uninstalling langchain-1.0.0:
  Successfully uninstalled langchain-1.0.0
Note: you may need to restart the kernel to use updated packages.


#### Cargar datos :

In [2]:
import feedparser
import pandas as pd


# URL del RSS de RPP
rss_url = "https://rpp.pe/rss"

feed = feedparser.parse(rss_url)

# Se extrae las 50 noticias más recientes
news_items = []
for entry in feed.entries[:50]:
    news_items.append({
        "title": entry.title,
        "description": entry.description,
        "link": entry.link,
        "date_published": entry.published if "published" in entry else None
    })

df_rpp = pd.DataFrame(news_items)
print(f"Noticias extraídas: {len(df_rpp)}")
df_rpp.head()


Noticias extraídas: 50


,title,description,link,date_published
0,LDU Quito vs. Palmeiras EN VIVO vía ESPN: se e...,Consulta todos los detalles de lo que será el ...,https://rpp.pe/futbol/copa-libertadores/ldu-qu...,"Thu, 23 Oct 2025 20:30:45 -0500"
1,Cúal fue el último temblor en México hoy 23 de...,Cuál es el ultimo temblor en México y CDMX reg...,https://rpp.pe/mundo/mexico/cual-fue-el-ultimo...,"Wed, 22 Oct 2025 06:42:35 -0500"
2,"André Martínez: ""Quiero ganar el Campeonato Mu...","André Martínez, ganador de Caminos del Inca 20...",https://rpp.pe/automovilismo/caminos-del-inca/...,"Thu, 23 Oct 2025 19:14:21 -0500"
3,Latin Billboard 2025: Karol G gana el primer p...,La cantante Karol G se llevó el galardón como ...,https://rpp.pe/musica/internacional/latin-bill...,"Thu, 23 Oct 2025 20:03:15 -0500"
4,Sporting Cristal vs Universitario HOY: ¿a qué ...,Cristal vs Universitario medirán fuerzas por l...,https://rpp.pe/futbol/descentralizado/universi...,"Thu, 23 Oct 2025 20:00:04 -0500"


 #### Tokenización :

In [ ]:
import tiktoken

# Elegimos un modelo de referencia
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Tomamos un artículo de ejemplo
sample_text = df_rpp["description"][0]
num_tokens = len(encoding.encode(sample_text))

print(f"Número de tokens en el primer artículo: {num_tokens}")

# Si el texto supera 1024 tokens, lo dividimos
def dividir_texto_si_excede(texto, max_tokens=1024):
    tokens = encoding.encode(texto)
    if len(tokens) <= max_tokens:
        return [texto]  
    else:
        partes = []
        for i in range(0, len(tokens), max_tokens):
            sub_tokens = tokens[i:i + max_tokens]
            sub_texto = encoding.decode(sub_tokens)
            partes.append(sub_texto)
        return partes

# Aplicamos la función al texto de ejemplo
if num_tokens > 1024:
    partes = dividir_texto_si_excede(sample_text)
    print(f"El texto se dividió en {len(partes)} fragmentos.")
else:
    print("No fue necesario dividir el texto.")

Número de tokens en el primer artículo: 32
No fue necesario dividir el texto.


 #### Generar incrustaciones (embeddings) :

In [4]:
from sentence_transformers import SentenceTransformer

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(model_name)

# Crear las incrustaciones
embeddings = embedder.encode(df_rpp["description"].tolist(), show_progress_bar=True)

print(f"Forma de las incrustaciones: {embeddings.shape}")


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Forma de las incrustaciones: (50, 384)


 #### Crear base de datos vectorial con ChromaDB

In [5]:
import chromadb
from chromadb.utils import embedding_functions

chroma_client = chromadb.Client()

# Crear o conectar a una colección
collection = chroma_client.get_or_create_collection(name="rpp_news")

# Insertar los documentos y sus metadatos
collection.add(
    documents=df_rpp["description"].tolist(),
    embeddings=embeddings,
    metadatas=[{
        "title": t,
        "link": l,
        "date_published": d
    } for t, l, d in zip(df_rpp["title"], df_rpp["link"], df_rpp["date_published"])],
    ids=[f"id_{i}" for i in range(len(df_rpp))]
)

print(" Noticias insertadas correctamente en ChromaDB")

 Noticias insertadas correctamente en ChromaDB


#### Consultas de similitud: 

In [6]:
query = "Últimas noticias de economía"

query_embedding = embedder.encode([query])
results = collection.query(
    query_embeddings=query_embedding,
    n_results=5
)

# Mostrar los resultados en un DataFrame
results_df = pd.DataFrame({
    "title": [m["title"] for m in results["metadatas"][0]],
    "description": results["documents"][0],
    "link": [m["link"] for m in results["metadatas"][0]],
    "date_published": [m["date_published"] for m in results["metadatas"][0]]
})

print("Resultados de la búsqueda por similitud:")
results_df


Resultados de la búsqueda por similitud:


,title,description,link,date_published
0,MEF y el presidente Jerí revisan el Presupuest...,"La ministra de Economía y Finanzas, Denisse Mi...",https://rpp.pe/economia/economia/mef-y-el-pres...,"Thu, 23 Oct 2025 19:18:25 -0500"
1,"Isabelle Tate, actriz de '9-1-1: Nashville', m...",Su agencia confirmó la noticia en redes social...,https://rpp.pe/famosos/celebridades/murio-isab...,"Thu, 23 Oct 2025 17:00:54 -0500"
2,"Tras cambio en el reglamento del Congreso, arc...",Expedientes abiertos por el personal de fiscal...,https://rpp.pe/politica/elecciones/tras-cambio...,"Thu, 23 Oct 2025 19:49:41 -0500"
3,Ministerio Público solicitó impedimento de sal...,La Fiscalía solicitó al Poder Judicial que dic...,https://rpp.pe/lima/judiciales/ministerio-publ...,"Thu, 23 Oct 2025 19:07:15 -0500"
4,Comisión de Energía y Minas plantea extensión ...,El predictamen de la Comisión de Energía y Min...,https://rpp.pe/politica/congreso/comision-de-e...,"Thu, 23 Oct 2025 18:59:11 -0500"


#### Orquestación con LangChain :


In [10]:
from langchain_core.prompts import PromptTemplate

# Solo simularemos un flujo completo (sin necesidad de API de OpenAI)
template = """
Dada la siguiente descripción de noticia: "{description}"
Responde con una categoría temática general (por ejemplo: política, deportes, economía, sociedad, tecnología).
"""
prompt = PromptTemplate(input_variables=["description"], template=template)

# Simulador de LLM simple
class SimpleFakeLLM:
    """Simulador de LLM que devuelve respuestas predefinidas en orden circular."""
    
    def __init__(self, responses):
        self.responses = responses
        self.index = 0
    
    def predict(self, prompt_text: str) -> str:
        """Devuelve la siguiente respuesta de la lista."""
        response = self.responses[self.index]
        self.index = (self.index + 1) % len(self.responses)
        return response

# Crear LLM simulado
fake_llm = SimpleFakeLLM(
    responses=["economía", "política", "sociedad", "tecnología", "deportes"]
)

# Pipeline: formatea prompt → envía a LLM → devuelve respuesta
def clasificar_noticia(description: str, prompt_template, llm) -> str:
    """Pipeline que formatea el prompt y obtiene predicción del LLM."""
    formatted_prompt = prompt_template.format(description=description)
    return llm.predict(formatted_prompt)

# Probar con una noticia
test_description = df_rpp["description"][1]
result = clasificar_noticia(test_description, prompt, fake_llm)

print(f" Noticia: {df_rpp['title'][1][:60]}...")
print(f" Categoría predicha: {result}")

# Clasificar todas las noticias
print("\n Clasificando todas las noticias...\n")
categorias = []
for idx, desc in enumerate(df_rpp["description"]):
    categoria = clasificar_noticia(desc, prompt, fake_llm)
    categorias.append(categoria)
    if idx < 5:
        print(f"{idx+1}. {df_rpp['title'][idx][:60]}... → {categoria}")

df_rpp["categoria_simulada"] = categorias

print(f"\n Distribución de categorías:")
print(df_rpp["categoria_simulada"].value_counts())

print("\n Código ejecutado correctamente")

 Noticia: Cúal fue el último temblor en México hoy 23 de octubre según...
 Categoría predicha: economía

 Clasificando todas las noticias...

1. LDU Quito vs. Palmeiras EN VIVO vía ESPN: se enfrentan por l... → política
2. Cúal fue el último temblor en México hoy 23 de octubre según... → sociedad
3. André Martínez: "Quiero ganar el Campeonato Mundial de Rally... → tecnología
4. Latin Billboard 2025: Karol G gana el primer premio de la no... → deportes
5. Sporting Cristal vs Universitario HOY: ¿a qué hora juegan en... → economía

 Distribución de categorías:
categoria_simulada
política      10
sociedad      10
tecnología    10
deportes      10
economía      10
Name: count, dtype: int64

 Código ejecutado correctamente
